In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from time import sleep

In [2]:
stock_symbols = pd.read_csv(r"/Users/harnishsavadia/Desktop/Capstone Project/data/stock_list.csv")

# we have a character that will cause issues in our request so we have to remove it
stock_symbols['Symbol'] = stock_symbols['Symbol'].str.replace('^','')

# display the number of rows.
display(stock_symbols.shape)
print(stock_symbols.head())

/var/folders/sp/kv_nm1x15xs4v4444j_8fyyh0000gn/T/ipykernel_7737/1455773717.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stock_symbols['Symbol'] = stock_symbols['Symbol'].str.replace('^','')


(497, 1)

  Symbol
0    MMM
1    AOS
2    ABT
3   ABBV
4    ACN


In [13]:
# Dictionary to store all stock data
stock_dict = {}

# Loop through each symbol and get the data
for symbol in stock_symbols['Symbol']:
    try:
        stock = yf.Ticker(symbol)
        
        # Get basic info including financial metrics
        info = stock.info
        
        # Store metrics in dictionary
        stock_dict[symbol] = {
            # 'Symbol': info.get('symbol'),
            'Book Value': info.get('bookValue'),
            'Current Ratio': info.get('currentRatio'),
            'Dividend Yield': info.get('dividendYield'),
            'Shares Outstanding': info.get('sharesOutstanding'),
            'Debt to Equity': info.get('debtToEquity'),
            'Volume 10D Avg': info.get('averageVolume10days'),
            'Volume 3M Avg': info.get('averageVolume'),
            'Return on Equity': info.get('returnOnEquity'),
            'Return on Assets': info.get('returnOnAssets'),
            'Return on Investment': info.get('returnOnCapital'),
            'P/E Ratio': info.get('trailingPE'),
            'Forward P/E': info.get('forwardPE'),
            'PEG Ratio': info.get('pegRatio'),
            'Free Cash Flow': info.get('freeCashflow'),
            'Operating Cash Flow': info.get('operatingCashflow'),
            'Levered Free Cash Flow': info.get('leveredFreeCashflow'),
            'Price to Cash Flow': info.get('priceToOperatingCashflows'),
            'Capital Expenditures': info.get('capitalExpenditures'),
        }
        
        # Add small delay to avoid rate limits
        sleep(0.1)
        
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        continue

# Create DataFrame from dictionary
stock_df = pd.DataFrame.from_dict(stock_dict, orient='index')

# Clean up the DataFrame
stock_df = stock_df.replace([np.inf, -np.inf], np.nan)  # Replace infinite values
stock_df = stock_df.round(2)  # Round numerical values

# Display first few rows
print("Stock Metrics DataFrame:")
print(stock_df.head())

# Display shape
print("\nDataFrame Shape:", stock_df.shape)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=hZ6MW5YNHTh
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=hZ6MW5YNHTh
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CDAY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CDAY&crumb=hZ6MW5YNHTh
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FLT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.c

Stock Metrics DataFrame:
     Symbol  Book Value  Current Ratio  Dividend Yield  Shares Outstanding  \
MMM     MMM        8.52           1.43            0.02        5.445590e+08   
AOS     AOS       13.22           1.67            0.02        1.191100e+08   
ABT     ABT       22.94           1.60            0.02        1.734460e+09   
ABBV   ABBV        3.41           0.64            0.04        1.767140e+09   
ACN     ACN       45.24           1.10            0.02        6.248560e+08   

      Debt to Equity  Volume 10D Avg  Volume 3M Avg  Return on Equity  \
MMM           295.44       3703630.0      3651652.0              1.13   
AOS             7.47       1072980.0       982217.0              0.30   
ABT            37.59       5543280.0      4920093.0              0.15   
ABBV         1174.82       6018580.0      4700869.0              0.56   
ACN            14.13       2597030.0      2261942.0              0.27   

      Return on Assets Return on Investment  P/E Ratio  Forward P/E

In [40]:
# define our indicators list
metrics_list = ['Return on Equity', 'Return on Assets', 
                'Dividend Yield', 'P/E Ratio', 'Free Cash Flow', 'Shares Outstanding' 
                ]

# select only those columns
indicators_df = stock_df[metrics_list]

# clustering can't handle missing values, so we need to eliminate any row that has a missing value.
indicators_df = indicators_df[indicators_df[metrics_list] != 0]
indicators_df = indicators_df.dropna(how='any')

indicators_df["Free Cash Flow"] = (indicators_df["Free Cash Flow"] / 1e9).round(2).astype(str) + " B"
indicators_df["Shares Outstanding"] = (indicators_df["Shares Outstanding"] / 1e6).round(2).astype(str) + " M"

display(indicators_df.head())

,Return on Equity,Return on Assets,Dividend Yield,P/E Ratio,Free Cash Flow,Shares Outstanding
MMM,1.13,0.09,0.02,13.825364,3.13 B,544.56 M
AOS,0.30,0.14,0.02,19.815304,0.4 B,119.11 M
ABT,0.15,0.06,0.02,35.501522,6.4 B,1734.46 M
ABBV,0.56,0.08,0.04,60.565968,17.74 B,1767.14 M
ACN,0.27,0.12,0.02,31.25984,7.64 B,624.86 M


In [49]:
# Define the filters for each indicator
roe_filter = indicators_df['Return on Equity'] < 40
roa_filter = indicators_df['Return on Assets'] < 30
dividend_yield_filter = indicators_df['Dividend Yield'] < 20
pe_ratio_filter = indicators_df['P/E Ratio'] < 3000
free_cash_flow_filter = indicators_df['Free Cash Flow'].apply(lambda x: float(x.replace(' B', '')) < 100)  # Convert B to number and apply threshold
shares_outstanding_filter = indicators_df['Shares Outstanding'].apply(lambda x: float(x.replace(' M', '')) < 10000)  # Convert M to number and apply threshold

# Get the counts for each column
roe_count = pd.Series(roe_filter).value_counts()
roa_count = pd.Series(roa_filter).value_counts()
dividend_yield_count = pd.Series(dividend_yield_filter).value_counts()
pe_ratio_count = pd.Series(pe_ratio_filter).value_counts()
free_cash_flow_count = pd.Series(free_cash_flow_filter).value_counts()
shares_outstanding_count = pd.Series(shares_outstanding_filter).value_counts()

# Display the results
display(roe_count)
display(roa_count)
display(dividend_yield_count)
display(pe_ratio_count)
display(free_cash_flow_count)
display(shares_outstanding_count)


True    297
Name: Return on Equity, dtype: int64

True    297
Name: Return on Assets, dtype: int64

True    297
Name: Dividend Yield, dtype: int64

True    297
Name: P/E Ratio, dtype: int64

True    297
Name: Free Cash Flow, dtype: int64

True    297
Name: Shares Outstanding, dtype: int64

In [53]:
# Filter the data frame based on the conditions for each indicator
indicators_df = indicators_df[
    roe_filter & roa_filter & dividend_yield_filter & pe_ratio_filter & 
    free_cash_flow_filter & shares_outstanding_filter
]

# Display the filtered results
display(indicators_df.head())


,Return on Equity,Return on Assets,Dividend Yield,P/E Ratio,Free Cash Flow,Shares Outstanding
MMM,1.13,0.09,0.02,13.825364,3.13 B,544.56 M
AOS,0.30,0.14,0.02,19.815304,0.4 B,119.11 M
ABT,0.15,0.06,0.02,35.501522,6.4 B,1734.46 M
ABBV,0.56,0.08,0.04,60.565968,17.74 B,1767.14 M
ACN,0.27,0.12,0.02,31.25984,7.64 B,624.86 M


In [58]:
# Create a statistical summary for the filtered indicators_df
desc_df = indicators_df.describe()

# # Add the +3_std and -3_std rows based on mean and std values
# desc_df.loc['+3_std'] = desc_df.loc['mean'] + (desc_df.loc['std'] * 3)
# desc_df.loc['-3_std'] = desc_df.loc['mean'] - (desc_df.loc['std'] * 3)

# Display the updated summary
display(desc_df)


,Return on Equity,Return on Assets,Dividend Yield
count,297.000000,297.000000,297.000000
mean,0.346330,0.065084,0.023064
std,0.895832,0.037857,0.013089
min,-0.030000,0.010000,0.010000
25%,0.100000,0.030000,0.010000
50%,0.180000,0.060000,0.020000
75%,0.300000,0.080000,0.030000
max,11.950000,0.200000,0.070000
